In [ ]:
import numpy as np
import pandas as pd
import serial

In [ ]:
# Format: Object-Distance-Height
class_names = ["Coin-10cm-10cm", "Coin-10cm-30cm", "Coin-30cm-10cm", "Coin-30cm-30cm"
               "Eraser-10cm-10cm", "Eraser-10cm-30cm", "Eraser-30cm-10cm", "Eraser-30cm-30cm"]

# Data Collection
Modify # Parameters

In [ ]:
# Using serial to read data from a serial port - using correct port
ser = serial.Serial(port="COM8", baudrate=230400, timeout=1) 
ser.flush()  # Flush the serial buffer

# Function to read ADC value from the serial port
def read_adc_value():
    try:
        if ser.in_waiting > 0:
            line = ser.readline().decode('utf-8').strip()
            return int(line)
    except ValueError:
        pass
    return None

# Function to save data to a CSV file
def save_to_csv(data, label, filename="data.csv"):
    df.loc[len(df)] = [label] + data
    df.to_csv(filename, index=False)

# Parameters
adc_threshold = 1000
num_items_to_collect = 100  # Number of items to collect
class_label = class_names.index("Coin-10cm-10cm")

# Read csv if it exists, else create a new one
try:
    df = pd.read_csv("data.csv")
except FileNotFoundError:
    df = pd.DataFrame(columns=["label"] + ["data" + str(i + 1) for i in range(num_items_to_collect)])

# Data collection
data = []

try:
    print("Data collection started...")
    activated = False  # Flag to check if threshold was crossed
    while len(data) < num_items_to_collect:
        adc_value = read_adc_value()
        if adc_value is not None:
            if not activated and adc_value > adc_threshold:
                activated = True  # Activate reading after threshold is crossed
                print("Threshold crossed. Collecting data...")
            if activated:
                # Collect data
                data.append(adc_value)  # Store ADC values as rows
    print("Data collection completed.")
except KeyboardInterrupt:
    print("Data collection interrupted.")
except Exception as e:
    print(f"An error occurred: {e}")

# Save data to CSV
if data:
    save_to_csv(data, class_label)
    print("Data saved to CSV.")
else:
    print("No data collected.")

ser.close()